In [1]:
import numpy as np
import tensorflow as tf

tf.reset_default_graph()
sess = tf.InteractiveSession()

PAD = 0
EOS = 1

In [2]:
x = [[2, 4, 8],
     [2, 5, 9],
     [3, 6, 0],
     [0, 7, 0]]
xl = [3, 4, 2]

encoder_inputs = tf.constant(x, dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.constant(xl, dtype=tf.int32, name='encoder_inputs_length')

In [4]:
import math

vocab_size = 10
embedding_size = 5
input_hidden_unit = 20

with tf.variable_scope("embedding") as scope:
    encoder_cell = tf.contrib.rnn.LSTMCell(input_hidden_unit)
    sqrt3 = math.sqrt(3)
    initializer = tf.random_uniform_initializer(-sqrt3, sqrt3)

    embedding_matrix = tf.get_variable(
        name="embedding_matrix",
        shape=[vocab_size, embedding_size],
        initializer=initializer,
        dtype=tf.float32)

    embedded_encoder_inputs = tf.nn.embedding_lookup(
        embedding_matrix, encoder_inputs)
    
    outputs, state = tf.nn.dynamic_rnn(
        cell=encoder_cell,
        inputs=embedded_encoder_inputs,
        sequence_length=encoder_inputs_length,
        time_major=True,
        dtype=tf.float32)

In [9]:
state.c

<tf.Tensor 'embedding/rnn/while/Exit_2:0' shape=(3, 20) dtype=float32>

In [ ]:
output_hidden_unit = 10

decoder_cell = tf.contrib.rnn.LSTMCell(output_hidden_unit)
helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
    embedding_decoder, start_tokens, end_token)

decoder = tf.contrib.seq2seq.BasicDecoder(
    cell=cell,
    helper=helper,
    initial_state=cell.zero_state(batch_size, tf.float32)
)

In [19]:
input_hidden_unit = 20
output_hidden_unit = 10

encoder_cell = tf.contrib.rnn.LSTMCell(input_hidden_unit)
decoder_cell = tf.contrib.rnn.LSTMCell(output_hidden_unit)

In [20]:
x = [[5, 6, 7],
     [7, 6, 8],
     [3, 8, 0],
     [0, 7, 0]]
xl = [3, 4, 2]
encoder_inputs = tf.constant(x, dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.constant(xl, dtype=tf.int32, name='encoder_inputs_length')

decoder_targets = tf.constant(x, dtype=tf.int32, name='decoder_targets')
decoder_targets_length = tf.constant(xl, dtype=tf.int32, name='decoder_targets_length')

In [4]:
with tf.name_scope('DecoderTrainFeeds'):
    sequence_size, batch_size = tf.unstack(tf.shape(decoder_targets))

    EOS_SLICE = tf.ones([1, batch_size], dtype=tf.int32) * EOS
    PAD_SLICE = tf.ones([1, batch_size], dtype=tf.int32) * PAD

    decoder_train_inputs = tf.concat([EOS_SLICE, decoder_targets], axis=0)
    decoder_train_length = decoder_targets_length + 1

    decoder_train_targets = tf.concat([decoder_targets, PAD_SLICE], axis=0)
    decoder_train_targets_seq_len, _ = tf.unstack(tf.shape(decoder_train_targets))
    decoder_train_targets_eos_mask = tf.one_hot(
        decoder_train_length - 1,
        decoder_train_targets_seq_len,
        on_value=EOS,
        off_value=PAD,
        dtype=tf.int32
    )
    decoder_train_targets_eos_mask = tf.transpose(
        decoder_train_targets_eos_mask, [1, 0])

    # hacky way using one_hot to put EOS symbol at the end of target sequence
    decoder_train_targets = tf.add(decoder_train_targets,
                                   decoder_train_targets_eos_mask)

    loss_weights = tf.ones([
        batch_size,
        tf.reduce_max(decoder_train_length)
    ], dtype=tf.float32, name="loss_weights")


In [24]:
import math

vocab_size = 10
embedding_size = 5

with tf.variable_scope("embeddingss") as scope:
    sqrt3 = math.sqrt(3)
    initializer = tf.random_uniform_initializer(-sqrt3, sqrt3)

    embedding_matrix = tf.get_variable(
        name="embedding_matrix",
        shape=[vocab_size, embedding_size],
        initializer=initializer,
        dtype=tf.float32)

    encoder_inputs_embedded = tf.nn.embedding_lookup(
        embedding_matrix, encoder_inputs)




In [30]:
encoder_inputs_length.shape

TensorShape([Dimension(3)])

In [25]:
with tf.variable_scope("Encoder") as scope:
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
        cell=encoder_cell,
        inputs=encoder_inputs_embedded,
        sequence_length=encoder_inputs_length,
        time_major=True,
        dtype=tf.float32
    )
        

In [22]:
tf.contrib.layers.OPTIMIZER_CLS_NAMES

{'Adagrad': tensorflow.python.training.adagrad.AdagradOptimizer,
 'Adam': tensorflow.python.training.adam.AdamOptimizer,
 'Ftrl': tensorflow.python.training.ftrl.FtrlOptimizer,
 'Momentum': tensorflow.python.training.momentum.MomentumOptimizer,
 'RMSProp': tensorflow.python.training.rmsprop.RMSPropOptimizer,
 'SGD': tensorflow.python.training.gradient_descent.GradientDescentOptimizer}

In [17]:
sess.run(encoder_inputs_length)

array([3, 4, 2], dtype=int32)

In [21]:
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        cell=deocder_cell,
        attention_mechanism=attention_mechanism,
        name='Attention_Wrapper'
    )
    

In [ ]:
with tf.variable_scope("Decoder") as scope:
    attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
        num_units=input_hidden_unit,
        memory=encoder_outputs
    )
    
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        cell=decoder_cell,
        attention_mechanism=attention_mechanism,
        name='Attention_Wrapper'
    )
    
    def output_fn(outputs):
        return tf.contrib.layers.linear(outputs, self.vocab_size, scope=scope)

    if not attention:
        decoder_fn_train = seq2seq.simple_decoder_fn_train(encoder_state=encoder_state)
        decoder_fn_inference = seq2seq.simple_decoder_fn_inference(
            output_fn=output_fn,
            encoder_state=encoder_state,
            embeddings=embedding_matrix,
            start_of_sequence_id=self.EOS,
            end_of_sequence_id=self.EOS,
            maximum_length=tf.reduce_max(self.encoder_inputs_length) + 3,
            num_decoder_symbols=self.vocab_size,
        )
    else:
        # attention_states: size [batch_size, max_time, num_units]
        attention_states = tf.transpose(self.encoder_outputs, [1, 0, 2])

        (attention_keys,
        attention_values,
        attention_score_fn,
        attention_construct_fn) = seq2seq.prepare_attention(
            attention_states=attention_states,
            attention_option="bahdanau",
            num_units=self.decoder_hidden_units,
        )

        decoder_fn_train = seq2seq.attention_decoder_fn_train(
            encoder_state=self.encoder_state,
            attention_keys=attention_keys,
            attention_values=attention_values,
            attention_score_fn=attention_score_fn,
            attention_construct_fn=attention_construct_fn,
            name='attention_decoder'
        )

        decoder_fn_inference = seq2seq.attention_decoder_fn_inference(
            output_fn=output_fn,
            encoder_state=self.encoder_state,
            attention_keys=attention_keys,
            attention_values=attention_values,
            attention_score_fn=attention_score_fn,
            attention_construct_fn=attention_construct_fn,
            embeddings=self.embedding_matrix,
            start_of_sequence_id=self.EOS,
            end_of_sequence_id=self.EOS,
            maximum_length=tf.reduce_max(self.encoder_inputs_length) + 3,
            num_decoder_symbols=self.vocab_size,
        )

    (self.decoder_outputs_train,
     self.decoder_state_train,
     self.decoder_context_state_train) = (
        seq2seq.dynamic_rnn_decoder(
            cell=self.decoder_cell,
            decoder_fn=decoder_fn_train,
            inputs=self.decoder_train_inputs_embedded,
            sequence_length=self.decoder_train_length,
            time_major=True,
            scope=scope,
        )
    )

    self.decoder_logits_train = output_fn(self.decoder_outputs_train)
    self.decoder_prediction_train = tf.argmax(self.decoder_logits_train, axis=-1, name='decoder_prediction_train')

    scope.reuse_variables()

    (self.decoder_logits_inference,
     self.decoder_state_inference,
     self.decoder_context_state_inference) = (
        seq2seq.dynamic_rnn_decoder(
            cell=self.decoder_cell,
            decoder_fn=decoder_fn_inference,
            time_major=True,
            scope=scope,
        )
    )
    self.decoder_prediction_inference = tf.argmax(self.decoder_logits_inference, axis=-1, name='decoder_prediction_inference')


In [ ]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

In [ ]:
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, decoder_inputs)

In [ ]:
encoder_cell = tf.contrib.rnn.LSTMCell(encoder_hidden_units)

encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs_embedded,
    dtype=tf.float32, time_major=True,
)

del encoder_outputs

In [ ]:
encoder_final_state

In [ ]:
decoder_cell = tf.contrib.rnn.LSTMCell(decoder_hidden_units)

decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
    decoder_cell, decoder_inputs_embedded,

    initial_state=encoder_final_state,

    dtype=tf.float32, time_major=True, scope="plain_decoder",
)

In [ ]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocab_size)

decoder_prediction = tf.argmax(decoder_logits, 2)

In [ ]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)

loss = tf.reduce_mean(stepwise_cross_entropy)

In [ ]:
sess.run(tf.global_variables_initializer())


In [ ]:
batch_ = [[6], [3, 4], [9, 8, 7]]

batch_, batch_length_ = batch(batch_)
print('batch_encoded:\n' + str(batch_))

din_, dlen_ = batch(np.ones(shape=(3, 1), dtype=np.int32),
                            max_sequence_length=4)
print('decoder inputs:\n' + str(din_))

pred_ = sess.run(decoder_prediction,
    feed_dict={
        encoder_inputs: batch_,
        decoder_inputs: din_,
    })
print('decoder predictions:\n' + str(pred_))